In [118]:
import pandas as pd
import os
import re
import sys
import ast
import csv

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


In [154]:
datasets = ["ace2004", "ace2005", "docred", "hlt","re3d", "tacred", "hlt"]
model_type = "Actual Relation"

In [155]:
#No Kbp37
et_dictionary = {
"hlt": {'ORG-ORG': 0,
'PER-LOC': 1,
'PER-ORG': 2}, 
"ace2004": {'GPE-GPE': 0,
 'GPE-LOC': 1,
 'GPE-ORG': 2,
 'GPE-PER': 3,
 'LOC-GPE': 4,
 'LOC-LOC': 5,
 'LOC-ORG': 6,
 'LOC-PER': 7,
 'ORG-GPE': 8,
 'ORG-LOC': 9,
 'ORG-ORG': 10,
 'ORG-PER': 11,
 'PER-GPE': 12,
 'PER-LOC': 13,
 'PER-ORG': 14,
 'PER-PER': 15},
"ace2005":{'GPE-GPE': 0,
 'GPE-LOC': 1,
 'GPE-ORG': 2,
 'GPE-PER': 3,
 'LOC-GPE': 4,
 'LOC-LOC': 5,
 'LOC-ORG': 6,
 'LOC-PER': 7,
 'ORG-GPE': 8,
 'ORG-LOC': 9,
 'ORG-ORG': 10,
 'ORG-PER': 11,
 'PER-GPE': 12,
 'PER-LOC': 13,
 'PER-ORG': 14,
 'PER-PER': 15}, 
"docred":{'LOC-LOC': 0,
 'LOC-ORG': 1,
 'LOC-PER': 2,
 'ORG-LOC': 3,
 'ORG-ORG': 4,
 'ORG-PER': 5,
 'PER-LOC': 6,
 'PER-ORG': 7,
 'PER-PER': 8}, 
"re3d":{'LOC-LOC': 0,
 'ORG-LOC': 1,
 'ORG-ORG': 2,
 'PER-LOC': 3,
 'PER-ORG': 4,
 'PER-PER': 5}, 
"tacred":{'LOC-PER': 0,
  'PER-LOC': 1,
  'PER-PER': 2}
}

In [156]:
actual_relation_dictionary = {
"ace2004" : {'Based-In': 0,
 'Business': 1,
 'Citizen-or-Resident': 2,
 'Employ-Executive': 3,
 'Employ-Staff': 4,
 'Employ-Undetermined': 5,
 'Ethnic': 6,
 'Family': 7,
 'Ideology': 8,
 'Located': 9,
 'Member-of-Group': 10,
 'Near': 11,
 'Other': 12,
 'Part-Whole': 13,
 'Subsidiary': 14,
 'no_relation': 15},
"ace2005" :{'Business': 0,
 'Citizen-Resident-Religion-Ethnicity': 1,
 'Employment': 2,
 'Family': 3,
 'Founder': 4,
 'Geographical': 5,
 'Investor-Shareholder': 6,
 'Located': 7,
 'Membership': 8,
 'Near': 9,
 'Org-Location': 10,
 'Ownership': 11,
 'Sports-Affiliation': 12,
 'Subsidiary': 13,
 'no_relation': 14},
"hlt":{'Aquisition': 0, 'Birthplace': 1},
    "docred":{'child': 0,
 'contains administrative territorial entity': 1,
 'continent': 2,
 'country': 3,
 'country of citizenship': 4,
 'educated at': 5,
 'employer': 6,
 'father': 7,
 'has part': 8,
 'head of government': 9,
 'head of state': 10,
 'headquarters location': 11,
 'located in or next to body of water': 12,
 'located in the administrative territorial entity': 13,
 'located on terrain feature': 14,
 'member of': 15,
 'member of political party': 16,
 'member of sports team': 17,
 'military branch': 18,
 'mouth of the watercourse': 19,
 'part of': 20,
 'performer': 21,
 'place of birth': 22,
 'place of death': 23,
 'record label': 24,
 'sibling': 25,
 'spouse': 26},
    "tacred":{'no_relation': 0,
 'per:alternate_names': 1,
 'per:children': 2,
 'per:cities_of_residence': 3,
 'per:city_of_death': 4,
 'per:employee_of': 5,
 'per:other_family': 6,
 'per:parents': 7,
 'per:siblings': 8,
 'per:spouse': 9,
 'per:stateorprovinces_of_residence': 10},
    "re3d":{'BelongsTo': 0, 'CoLocated': 1, 'CommWith': 2, 'FightingAgainst': 3}
}

In [157]:
binary_dictionary = {
    "No Relation": 0, 
    "Relation" : 1
}

In [164]:
base_path = "/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/"+model_type+"/"
with open("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/"+model_type+"_prediction_report.csv", "w") as f:
        
    for dataset in datasets:
        print(dataset)
        if model_type == "ET":
            dictionary = et_dictionary[dataset]
        elif model_type == "Binary":
            dictionary = binary_dictionary
        else:
            dictionary = actual_relation_dictionary[dataset]
        
        dictionary = {v: k for k, v in dictionary.items()}

        features = pd.read_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_"+dataset+".csv", index_col=0)
        prediction_data = pd.DataFrame()
        try:
            os.chdir(base_path+dataset)
            for file in os.listdir():
                data = pd.DataFrame()
                if file.endswith(".csv"):
                    data = pd.read_csv(file, header=0)
                    file_list = file.split("_")
                    if len(file_list) > 3:
                        file_name = r=f"{file_list[1]}_{file_list[2]}"
                    else:
                        file_name = file_list[1]

                    data["prediction"] = data["prediction"].apply(lambda x : dictionary[x])
                    if prediction_data.empty:
                        prediction_data = data
                        prediction_data = prediction_data.rename(columns = {"prediction" :file_name+"_prediction"})
                    else:
                        prediction_data[file_name + "_prediction"] = data["prediction"]

            if 'Unnamed: 0' in prediction_data.keys():
                prediction_data = prediction_data.drop(["Unnamed: 0"], axis = 1) 
            features_reduced = features.drop_duplicates(subset=['sentence','entity 1 name', "entity 2 name", "root_word", "root_index", "sentence_length"])
            df = pd.merge(prediction_data, features_reduced, on=['entity 1 name', "entity 2 name", "root_word", "root_index", "sentence_length"], how='left', suffixes=[None,'_2'])
            df = df[['sentence','sentence_length', 'root_word', 'root_index', 'entity_1_index',
           'entity_2_index', 'entity_distance', 'no_words_before_entity_1',
           'no_words_after_entity_2', 'entity_1_root_distance',
           'entity_2_root_distance', 'entity 1 name', 'entity 2 name',
           'entity_type_1', 'entity_type_2', 'entity_pos_1', 'entity_pos_2',
           'entity_dep_1', 'entity_dep_2', 'shortest_distance', 'label',
           'adaboost_prediction', 'logisitic_regression_prediction',
           'random_forest_prediction', 'svm_prediction', 'svm_rbf_prediction',
           'xgboost_prediction', 'svm_poly_prediction']]
            df["sentence"] = df["sentence"].apply(lambda x : " ".join(ast.literal_eval(x)))
            if model_type == "Binary":
                df["label"] = df["label"].apply(lambda x : dictionary[x])
            df.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/sentence_pred/" + model_type + "/" + dataset + "_all_predictions.csv", index=False)

        except:
            print("Error : ", sys.exc_info())
            # print("File not present : " + base_path + dataset)

hlt
